# What is selfrecflection in RAG
Self-Reflection = LLM evaluates its own output: "Is this clear, complete and accurate?"

1. Generates an initial answer using retrieved context
2. Reflects on that answer with a dedicated self-critic LLM step
3. If unsatisfied, it can revise the query, retrieve again, or regenerate the answer

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import Tool, tool, StructuredTool
from langchain_core.documents import Document
from langchain.agents import create_agent
from pydantic import BaseModel
from typing import List
from langgraph.graph import StateGraph, END

In [16]:
docs = TextLoader(
    "research_notes.txt",
    encoding="utf-8"
).load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

In [17]:
llm = ChatOllama(model="qwen3:4b")

In [18]:
class RAGReflectionState(BaseModel):
    question: str
    retrieved_docs: List[Document] = []
    answer: str = ""
    reflection: str = ""
    revised: bool = False
    attempts: int = 0

In [19]:
def retrieve_docs(state: RAGReflectionState) -> RAGReflectionState:
    docs = retriever.invoke(state.question)
    return state.model_copy(update={"retrieved_docs": docs})

def generate_answer(state: RAGReflectionState) -> RAGReflectionState:
    context  = "\n\n".join([doc.page_content for doc in state.retrieved_docs])
    prompt = f"""
        Use the follow context to answer the question:
        
        Context:
        {context}
        
        Question:
        {state.question}
    """
    
    answer = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"answer": answer, "attempts": state.attempts + 1})

In [20]:
def reflect_answer(state: RAGReflectionState) -> RAGReflectionState:
    
    prompt = f"""
       Reflect on the following answer to see if it fully addresses the question:
       Satte YES if it is complete and correct, or NO with an explanation.
       
       Question: {state.question}
       
       Answer: {state.answer}
       
       REspond like:
       Reflection: YES or NO
       
       Explanations: ...
    """
    
    result = llm.invoke(prompt).content
    is_ok = "reflection: yes" in result.lower()
    return state.model_copy(update={"reflection": result, "revised": not is_ok})

In [21]:
def finalize(state: RAGReflectionState) -> RAGReflectionState:
    return state

In [22]:
builder = StateGraph(RAGReflectionState)

builder.add_node("retriever", retrieve_docs)
builder.add_node("responder", generate_answer)
builder.add_node("reflector", reflect_answer)
builder.add_node("done", finalize)

builder.set_entry_point("retriever")

builder.add_edge("retriever", "responder")
builder.add_edge("responder", "reflector")
builder.add_conditional_edges(
    "reflector",
    lambda s: "done" if not s.revised or s.attempts >=2 else "retriever"
)
builder.add_edge("done", END)

graph = builder.compile()

In [24]:
user_query = "What are the benefits of agnet loops in AI?"
init_state = RAGReflectionState(question=user_query)
result = graph.invoke(init_state)

print("\n Final Answer:\n", result['answer'])
print("\n Reflection Log:\\n", result['reflection'])
print("Total Attempts:", result['attempts'])


 Final Answer:
 Based **strictly on the provided context**, there is **no mention of "agnet loops"** in the text. This appears to be a **typo or misstatement** (likely meant to be "agent loops" or "agent systems"). The context describes several AI systems and evaluation protocols but **does not discuss "agnet loops" at all**.

However, I can address what *is* relevant in the context to clarify your likely intent:

---

### 🔍 Key Clarification & What the Context *Does* Say About Agent Systems:
1. **The context mentions "retrieval-agent reasoning"** (Section 3):
   - This refers to **agent-based systems** that chain tools (e.g., SQL queries + Wikipedia) for dynamic reasoning.
   - **Benefit**: Enables multi-step workflows (e.g., "Give me customer insights from SQL, verify with wiki") by combining tools in a logical sequence.

2. **No "agnet loops" are described**:
   - The term "agnet" is not used in AI literature or this context.
   - The context discusses **agent systems** (e.g., retr